In [1]:
## DO NOT ERASE THIS. IMPORTANT TO CORRECTLY IMPORT MODULES
import sys

sys.path.append("../")
sys.executable

'/Users/kristina/Desktop/University/COURSE_WORK/RecSys_thesis/recsysvenv/bin/python3.12'

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# from scipy.sparse import csr_matrix
# from scipy.sparse.linalg import svds
# from surprise import SVDpp, Reader, Dataset
# from surprise.model_selection import cross_validate

from typing import Union
from tqdm.notebook import tqdm

import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch

from src.metrics import reccomendation_report
from src.utils import surprise_predict
from src.utils import split_test_df
from src.utils import seed_everything
from src.utils import trainDataset
from src.utils import trainDatasetWithNumCatFeatures

from src.models import DeepFM, DeepFMImp

import mmh3

## Constants

In [3]:
DATA_FOLDER = "../../data/ml-1m/"
RANDOM_STATE = 7

In [4]:
seed_everything(RANDOM_STATE)

## Data

In [5]:
df_movies = pd.read_csv(
    DATA_FOLDER + "movies.csv",
    encoding="iso-8859-1",
    sep=";",
    names=["movieId", "name", "genre"],
)
df_ratings = pd.read_csv(
    DATA_FOLDER + "ratings.csv",
    encoding="iso-8859-1",
    sep=";",
    names=["userId", "movieId", "rating", "timestamp"],
)
df_users = pd.read_csv(
    DATA_FOLDER + "users.csv",
    encoding="iso-8859-1",
    sep=";",
    names=["userId", "gender", "age", "occupation", "zip-code"],
)

In [6]:
## Encode usedId, movieId
user_encoder = LabelEncoder()
movie_encoder = LabelEncoder()

df_movies["movieId"] = movie_encoder.fit_transform(df_movies["movieId"])
df_users["userId"] = user_encoder.fit_transform(df_users["userId"])

df_ratings["movieId"] = movie_encoder.transform(df_ratings["movieId"])
df_ratings["userId"] = user_encoder.transform(df_ratings["userId"])

#### Creating a test_user

In [7]:
df_users.loc[-1] = [df_users["userId"].max() + 1, "F", 21, 5, 777777]
df_users = df_users.reset_index(drop=True)
df_test_user = pd.DataFrame(columns=["userId", "movieId", "rating", "timestamp"])
df_test_user = df_test_user.assign(movieId=[6, 16, 3192, 1461, 827, 887, 593]).assign(
    userId=df_users["userId"].max(), rating=4, timestamp=lambda x: np.arange(x.shape[0])
)
print("Test user watch list:")
display(df_test_user.merge(df_movies, on="movieId"))
df_ratings = pd.concat([df_ratings, df_test_user], ignore_index=True)

Test user watch list:


,userId,movieId,rating,timestamp,name,genre
0,6040,6,4,0,Sabrina (1995),Comedy|Romance
1,6040,16,4,1,Sense and Sensibility (1995),Drama|Romance
2,6040,3192,4,2,Singles (1992),Comedy|Drama|Romance
3,6040,1461,4,3,Love and Other Catastrophes (1996),Romance
4,6040,827,4,4,Emma (1996),Comedy|Drama|Romance
5,6040,887,4,5,Singin' in the Rain (1952),Musical|Romance
6,6040,593,4,6,Pretty Woman (1990),Comedy|Romance


### Train-test split
Methodology: Last user interaction is a test item. The rest is train. Validation part is 10% of test.

In [8]:
df_ratings["rank"] = (
    df_ratings[["userId", "timestamp"]]
    .groupby("userId", as_index=False)["timestamp"]
    .rank(method="first", ascending=False)
)

In [9]:
# leave one out
df_train = df_ratings.loc[df_ratings["rank"] != 1].reset_index(drop=True)
df_test = (
    df_ratings.loc[df_ratings["rank"] == 1].reset_index(drop=True).assign(action=1)
)
df_test, df_val = train_test_split(df_test, test_size=0.2, random_state=RANDOM_STATE)

In [10]:
# enrich test data with 100 random movies from the ones not intercated by user
df_add = pd.DataFrame()
for user in tqdm(df_test.userId.unique(), desc="Enriching test"):
    movie = df_test.loc[df_test.userId == user, "movieId"]
    watched_movies = np.append(
        movie, df_train.loc[df_train.userId == user, "movieId"].values
    )
    not_wathed_movies = np.setdiff1d(
        np.arange(df_movies["movieId"].max() + 1), watched_movies
    )
    random_100 = np.random.choice(not_wathed_movies, 100, replace=False)

    df_temp = pd.DataFrame().assign(movieId=random_100, userId=user, action=0)
    df_add = pd.concat([df_add, df_temp], ignore_index=True)

df_test = pd.concat([df_test, df_add], ignore_index=True).drop(
    columns=["timestamp", "rating", "rank"]
)

df_add = pd.DataFrame()
for user in tqdm(df_val.userId.unique(), desc="Enriching val"):
    movie = df_val.loc[df_val.userId == user, "movieId"]
    watched_movies = np.append(
        movie, df_train.loc[df_train.userId == user, "movieId"].values
    )
    not_wathed_movies = np.setdiff1d(
        np.arange(df_movies["movieId"].max() + 1), watched_movies
    )
    random_100 = np.random.choice(not_wathed_movies, 100, replace=False)

    df_temp = pd.DataFrame().assign(movieId=random_100, userId=user, action=0)
    df_add = pd.concat([df_add, df_temp], ignore_index=True)

df_val = pd.concat([df_val, df_add], ignore_index=True).drop(
    columns=["timestamp", "rating", "rank"]
)

# adding data for test user
df_add = pd.DataFrame()
user = df_users["userId"].max()
movie = df_test.loc[df_test.userId == user, "movieId"]
watched_movies = np.append(
    movie, df_train.loc[df_train.userId == user, "movieId"].values
)
not_wathed_movies = np.setdiff1d(
    np.arange(df_movies["movieId"].max() + 1), watched_movies
)
random_500 = np.random.choice(not_wathed_movies, 500, replace=False)

df_temp = pd.DataFrame().assign(movieId=random_500, userId=user, action=0)
df_add = pd.concat([df_add, df_temp], ignore_index=True)
df_test_user = df_add

Enriching test:   0%|          | 0/4832 [00:00<?, ?it/s]

Enriching val:   0%|          | 0/1209 [00:00<?, ?it/s]

In [11]:
seed_everything(RANDOM_STATE)
# enriching train with negative samples
users, items, labels = [], [], []
all_movieIds = df_movies["movieId"].nunique()
user_item_set = set(zip(df_train["userId"], df_train["movieId"]))

num_negatives = 4
for u, i in tqdm(user_item_set):
    users.append(u)
    items.append(i)
    labels.append(1)
    for _ in range(num_negatives):
        negative_item = np.random.choice(all_movieIds)
        while (u, negative_item) in user_item_set:
            negative_item = np.random.choice(all_movieIds)
        users.append(u)
        items.append(negative_item)
        labels.append(0)
df_train = pd.DataFrame(
    np.array([users, items, labels]).T, columns=["userId", "movieId", "label"]
)

  0%|          | 0/994175 [00:00<?, ?it/s]

In [12]:
with open("../../data/cos_dist.pt", "rb") as f:
    cos_dist = torch.load(f)
popularity = torch.bincount(
    torch.tensor(df_train.movieId.values), minlength=df_movies["movieId"].nunique()
)
popularity = popularity / torch.max(popularity)

## Building one-hot-encoded featurues and normalizing continious features

In [13]:
ord_user = OrdinalEncoder()
user_cat = pd.DataFrame(
    ord_user.fit_transform(df_users[["gender", "occupation"]]),
    columns=ord_user.feature_names_in_,
).assign(userId=df_users["userId"])

ord_movie = OrdinalEncoder()
movie_cat = pd.DataFrame(
    ord_movie.fit_transform(df_movies[["genre"]]), columns=ord_movie.feature_names_in_
).assign(movieId=df_movies["movieId"])

ss_user = StandardScaler()
user_num = pd.DataFrame(
    ss_user.fit_transform(df_users[["age"]]), columns=ss_user.feature_names_in_
).assign(userId=df_users["userId"])

In [14]:
df_train = (
    df_train[["userId", "movieId", "label"]]
    .merge(user_cat, on="userId")
    .merge(user_num, on="userId")
    .merge(movie_cat, on="movieId")
)

In [15]:
# test_user_cat = user_cat[df_test["userId"].values].clone().detach().to(torch.long)
# test_user_num = user_num[df_test["userId"].values].clone().detach().to(torch.float)
# test_movie_cat = movie_cat[df_test["movieId"].values].clone().detach().to(torch.long)
# test_cat = torch.hstack((test_user_cat, test_movie_cat))

# test_loader = DataLoader(
#     TensorDataset(
#         torch.tensor(df_test["userId"]),
#         torch.tensor(df_test["movieId"]),
#         test_user_num,
#         test_cat,
#     ),
#     batch_size=4096,
#     shuffle=False,
# )

## AutoInt from pytorch tabular

In [16]:
from pytorch_tabular import TabularModel
from pytorch_tabular.models import AutoIntConfig
from pytorch_tabular.config import (
    DataConfig,
    OptimizerConfig,
    TrainerConfig,
    ExperimentConfig,
)
from pytorch_tabular.models.common.heads import LinearHeadConfig

In [17]:
data_config = DataConfig(
    target=["label"],  # target should always be a list.
    continuous_cols=["age"],
    categorical_cols=["userId", "movieId", "gender", "occupation", "genre"],
)

trainer_config = TrainerConfig(
    accelerator="cpu",
    devices=10,
    auto_lr_find=False,  # Runs the LRFinder to automatically derive a learning rate
    batch_size=1024,
    max_epochs=10,
    early_stopping="valid_loss",  # Monitor valid_loss for early stopping
    early_stopping_mode="min",  # Set the mode as min because for val_loss, lower is better
    early_stopping_patience=5,  # No. of epochs of degradation training will wait before terminating
    checkpoints="valid_loss",  # Save best checkpoint monitoring val_loss
    load_best=True,  # After training, load the best checkpoint
)
optimizer_config = OptimizerConfig()

In [18]:
model_config = AutoIntConfig(
    task="classification",
    learning_rate=5e-4,
    head="LinearHead",  # Linear Head
)

In [19]:
seed_everything(RANDOM_STATE)
tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)
tabular_model.fit(train=df_train)

2024-04-21 19:05:30,855 - {pytorch_tabular.tabular_model:140} - INFO - Experiment Tracking is turned off

Seed set to 42


2024-04-21 19:05:30,869 - {pytorch_tabular.tabular_model:524} - INFO - Preparing the DataLoaders

2024-04-21 19:05:31,182 - {pytorch_tabular.tabular_datamodule:499} - INFO - Setting up the datamodule for          
classification task

/Users/kristina/Desktop/University/COURSE_WORK/RecSys_thesis/recsysvenv/lib/python3.12/site-packages/pytorch_tabular/categorical_encoders.py:67: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_encoded[col].fillna(self._imputed, inplace=True)
/Users/kristina/Desktop/University/COURSE_WORK/RecSys_thesis/recsysvenv/lib/python3.12/site-packages/pytorch_tabular/categorical_encoders.py:67: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior wi

...

/Users/kristina/Desktop/University/COURSE_WORK/RecSys_thesis/recsysvenv/lib/python3.12/site-packages/pytorch_lightn
ing/trainer/connectors/logger_connector/result.py:433: It is recommended to use `self.log('valid_accuracy', ..., 
sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.

/Users/kristina/Desktop/University/COURSE_WORK/RecSys_thesis/recsysvenv/lib/python3.12/site-packages/pytorch_lightn
ing/trainer/connectors/logger_connector/result.py:433: It is recommended to use `self.log('train_accuracy', ..., 
sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.

`Trainer.fit` stopped: `max_epochs=10` reached.


2024-04-21 19:19:45,014 - {pytorch_tabular.tabular_model:663} - INFO - Training the model completed

2024-04-21 19:19:45,019 - {pytorch_tabular.tabular_model:1489} - INFO - Loading the best model

In [20]:
df_test = (
    df_test[["userId", "movieId", "action"]]
    .merge(user_cat, on="userId")
    .merge(user_num, on="userId")
    .merge(movie_cat, on="movieId")
    .rename(columns={"action": "label"})
)

In [21]:
df_test["rating_pred"] = tabular_model.predict(df_test)["1_probability"]
pred, target, pred_items = split_test_df(
    df_test, "userId", "movieId", "rating_pred", "label"
)
reccomendation_report(pred, target, pred_items, cos_dist, popularity, k=15)

/Users/kristina/Desktop/University/COURSE_WORK/RecSys_thesis/recsysvenv/lib/python3.12/site-packages/pytorch_tabular/categorical_encoders.py:67: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_encoded[col].fillna(self._imputed, inplace=True)


{'Hit rate @ K': tensor(0.6207),
 'NDCG @ K': tensor(0.3133),
 'Diversity (ILD)': tensor(0.1347),
 'Novelty (EPC)': tensor(0.5386)}

### Test user reccomendations

In [34]:
pd.set_option("display.max_rows", 102)

df_test_user["action"] = df_test_user["label"]

In [35]:
df_test_user = (
    df_test_user[["userId", "movieId", "action"]]
    .merge(user_cat, on="userId")
    .merge(user_num, on="userId")
    .merge(movie_cat, on="movieId")
    .rename(columns={"action": "label"})
)
df_test_user["rating_pred"] = tabular_model.predict(df_test_user)["1_probability"]

display(
    df_test_user.loc[df_test_user.userId == df_test_user["userId"].max(), ["userId", "movieId", "rating_pred"]]
    .sort_values(by="rating_pred", ascending=False)
    .merge(df_movies, on="movieId")
    .loc[:100, ["userId", "movieId", "name", "genre"]]
)

/Users/kristina/Desktop/University/COURSE_WORK/RecSys_thesis/recsysvenv/lib/python3.12/site-packages/pytorch_tabular/categorical_encoders.py:67: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_encoded[col].fillna(self._imputed, inplace=True)


,userId,movieId,name,genre
0,6040,315,"Shawshank Redemption, The (1994)",Drama
1,6040,2728,Big (1988),Comedy|Fantasy
2,6040,1575,L.A. Confidential (1997),Crime|Film-Noir|Mystery|Thriller
3,6040,49,"Usual Suspects, The (1995)",Crime|Thriller
4,6040,1372,Jerry Maguire (1996),Drama|Romance
5,6040,847,"Godfather, The (1972)",Action|Crime|Drama
6,6040,3412,High Fidelity (2000),Comedy
7,6040,1227,"Graduate, The (1967)",Drama|Romance
8,6040,220,Clerks (1994),Comedy
9,6040,2252,Pleasantville (1998),Comedy
